In [1]:
# load relevant packages

import pandas as pd
import ast
import pysam
from Bio import SeqIO
import glob
import math
import numpy as np
from itertools import combinations
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import os
from dnds import dnds, pnps
import scipy.stats as stats
from scipy.stats import shapiro
import re

from utils import *
import argparse

In [2]:
# set up directories to which to point

experiment_id = 'jh_004' # user sets this

stats_df_paths = f'data/{experiment_id}/stats/'

csv_files = glob.glob(os.path.join(stats_df_paths, f"*cons.csv"))
print(csv_files)

['data/jh_004/stats/2025-09-04_stats_cons.csv']


In [3]:
stats_df = pd.read_csv(csv_files[0])
# print(stats_df)
stats_important_df = stats_df[['Sample', 'Gate', '%Gated']].copy()
print(stats_important_df)

           Sample        Gate   %Gated
0     29_1400nMAg  All Events  100.000
1     29_1400nMAg          R1   99.315
2     29_1400nMAg          R2   93.459
3     29_1400nMAg          R1   99.315
4     29_1400nMAg          R3   54.552
...           ...         ...      ...
1495    31ni_NoAg      +/+(3)    0.020
1496    31ni_NoAg      -/-(3)   99.695
1497    31ni_NoAg      +/-(3)    0.080
1498    31ni_NoAg          R8    0.000
1499    31ni_NoAg          R3   66.218

[1500 rows x 3 columns]


In [4]:
sample_list = list(set(stats_important_df['Sample']))
print(len(sample_list))



50


In [5]:
samples_dict = {}
cell_name_list = []
ag_conc_list =[]
for sample in sample_list:
    sample_split = sample.split('_')
    if 'ni' not in sample:
        cell_name_list.append(sample_split[0])
        ag_conc_list.append(sample_split[1])
    
    sample_stats_df = stats_important_df[stats_important_df['Sample'] == sample]
    # print(sample_stats_df)
    sample_pivot_df = sample_stats_df.pivot_table(
        index = 'Sample'
        , columns = 'Gate'
        , values = '%Gated'
        , aggfunc = 'first'
    )

    quads_gates = ['-/-(1)','-/+(1)','+/-(1)','+/+(1)','R6']

    
    sample_quad_df = sample_pivot_df[quads_gates].copy()
    sample_quad_df['onbindnorm'] = sample_quad_df['R6'] / (sample_quad_df['-/+(1)']+sample_quad_df['+/+(1)'])

    if 'NoAg' in sample:
        sample_quad_df['onbindnorm'] = sample_quad_df['+/+(1)'] / (sample_quad_df['-/+(1)']+sample_quad_df['+/+(1)'])
    
    sample_quad_df['sticky'] = sample_quad_df['+/-(1)'] / (sample_quad_df['+/-(1)']+sample_quad_df['-/-(1)']+sample_quad_df['-/+(1)'])
    samples_dict[sample] = sample_quad_df
    
# print(samples_dict)
print(list(set(cell_name_list)))
print(list(set(ag_conc_list)))

['35', '29', '30', '32', '36', '31', '37', '33', '34']
['350nMAg', '85nMAg', 'NoAgC', 'NoAg', '1400nMAg']


In [6]:
# print(cell_name_list)
for cellnum in list(set(cell_name_list)):
    print(cellnum)
    cell_keys = [key for key in samples_dict.keys() if cellnum + '_' in key and 'ni' not in key]
    # print(cell_keys)
    
    cell_subset_dict = {key: samples_dict[key] for key in cell_keys if key in samples_dict}
    
    cell_num_df= pd.concat(
        [df.assign(rep_key = key) for key, df in cell_subset_dict.items()],
        ignore_index=True)

    NoAgbind = 0
    
    for conc in cell_num_df['rep_key']:
        if 'NoAg' in conc:
            conc_slice = cell_num_df[cell_num_df['rep_key'] == conc]
            # print(conc_slice)
            NoAgbind = float(conc_slice['onbindnorm'].item() - conc_slice['sticky'].item())
            if NoAgbind < 0:
                NoAgbind = 0
            # print(conc_slice)
    print(f'NoAg binding correction for yJH{cellnum}: {NoAgbind}')
                
    cell_num_df['correctedbinding'] = cell_num_df['onbindnorm'] - cell_num_df['sticky'] - (NoAgbind)


    for idx in range(len(cell_num_df)): # set to 0 if corrected value is negative
        cell_num_df.loc[idx,'cell_name'] = 'yJH'+cellnum
        cell_num_df.loc[idx,'concentration'] = cell_num_df.loc[idx,'rep_key'].split('_')[1]
        numbers = ''.join(filter(str.isdigit, cell_num_df.loc[idx,'concentration']))
        cell_num_df.loc[idx,'concentration'] = int(numbers) if numbers else 0
        if cell_num_df.loc[idx,'correctedbinding'] < 0:
            cell_num_df.loc[idx,'correctedbinding'] = 0

    cell_num_df = cell_num_df.sort_values('concentration', ascending = False)

    print(cell_num_df[cell_num_df['correctedbinding'] > 0])
    
    bindcount_df_path = f'data/{experiment_id}/{cellnum}_bindingnorm.csv'
    cell_num_df.to_csv(bindcount_df_path)


35
NoAg binding correction for yJH35: 0.000308851386734932
Gate  -/-(1)  -/+(1)  +/-(1)  +/+(1)     R6  onbindnorm   sticky      rep_key  \
4     19.691  80.069   0.011   0.229  0.282    0.003512  0.00011  35_1400nMAg   
3     18.435  81.538   0.001   0.025  0.046    0.000564  0.00001    35_85nMAg   

Gate  correctedbinding cell_name concentration  
4             0.003093     yJH35          1400  
3             0.000245     yJH35            85  
29
NoAg binding correction for yJH29: 4.5271939588960856e-05
Gate  -/-(1)  -/+(1)  +/-(1)  +/+(1)     R6  onbindnorm   sticky      rep_key  \
3     18.185  81.770   0.008   0.037  0.047    0.000575  0.00008  29_1400nMAg   
0     18.126  81.861   0.000   0.013  0.021    0.000256  0.00000   29_350nMAg   
1     18.127  81.853   0.001   0.019  0.033    0.000403  0.00001    29_85nMAg   
2     17.811  82.173   0.001   0.015  0.034    0.000183  0.00001      29_NoAg   

Gate  correctedbinding cell_name concentration  
3             0.000449     yJH29  